In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [2]:
df = pd.read_csv('/Users/haoli/Downloads/df_for_modeling.csv')

In [3]:
df.head()

,id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,grade,sub_grade,emp_length,annual_inc,...,addr_state_freq,earliest_cr_line_len,f,w,months_to_last_pymnt,months_to_last_credit_pull,Individual,Joint App,N,Y
0,75710736,12800.0,12800.0,12800.0,11.99,425.09,3,31,6,53000.0,...,21752,177,0,1,44,45,1,0,1,0
1,76143686,9000.0,9000.0,9000.0,15.31,313.36,3,35,9,52000.0,...,40051,205,0,1,21,41,1,0,1,0
2,76143234,8000.0,8000.0,8000.0,10.75,260.97,2,24,5,73000.0,...,40051,143,0,1,45,43,1,0,1,0
3,75943579,20000.0,20000.0,20000.0,12.99,673.79,3,32,10,58000.0,...,7762,151,0,1,14,47,1,0,1,0
4,75169662,6000.0,6000.0,6000.0,7.39,186.34,1,14,3,72000.0,...,21752,98,0,1,21,21,1,0,1,0


In [4]:
df.drop('id', axis = 1, inplace = True)

In [5]:
train = df[df['issued_mark'] == 0]

In [6]:
test = df[df['issued_mark'] == 1]

In [7]:
train.shape

(235720, 114)

In [8]:
test.shape

(53118, 114)

In [9]:
train_y = train['y']
test_y = test['y']

In [10]:
train_x = train.drop('y', axis = 1)
test_x = test.drop('y', axis = 1)

In [11]:
train_x.drop('issued_mark', axis = 1, inplace = True)
test_x.drop('issued_mark', axis = 1, inplace = True)

In [12]:
train_y.value_counts()

0    197066
1     38654
Name: y, dtype: int64

In [13]:
#It is imbalanced, we do an undersampling here.
train_fully_paid = train[train['y'] == 0]

In [14]:
train_fully_paid['y'].value_counts()

0    197066
Name: y, dtype: int64

In [17]:
train_sample = train_fully_paid.sample(38654)

In [18]:
train_charged_off = train[train['y'] == 1]

In [19]:
train_new = pd.concat([train_sample, train_charged_off])

In [20]:
train_new.drop('issued_mark', axis = 1, inplace = True)

In [21]:
train_y = train_new['y']
train_x = train_new.drop('y', axis = 1)

In [22]:
train_y.value_counts()

1    38654
0    38654
Name: y, dtype: int64

In [56]:
rtc3 = RandomForestClassifier(max_depth = 3)

In [57]:
rtc3.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [26]:
from sklearn import metrics

In [59]:
print(metrics.classification_report(test_y, rtc3.predict(test_x)))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98     42154
           1       0.92      0.95      0.93     10964

    accuracy                           0.97     53118
   macro avg       0.95      0.96      0.96     53118
weighted avg       0.97      0.97      0.97     53118



In [60]:
rtc5 = RandomForestClassifier(max_depth = 5)

In [61]:
rtc5.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [63]:
print(metrics.classification_report(test_y, rtc5.predict(test_x)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     42154
           1       0.95      0.96      0.96     10964

    accuracy                           0.98     53118
   macro avg       0.97      0.97      0.97     53118
weighted avg       0.98      0.98      0.98     53118



In [15]:
from sklearn.model_selection import train_test_split

In [23]:
train_model, train_valid = train_test_split(train_new, train_size = 0.75)

In [24]:
train_model_x = train_model.drop('y', axis = 1)
train_model_y = train_model['y']

In [25]:
train_valid_x = train_valid.drop('y', axis = 1)
train_valid_y = train_valid['y']

In [27]:
rtc3 = RandomForestClassifier(max_depth = 3)

In [28]:
rtc3.fit(train_model_x, train_model_y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=3, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
valid = rtc3.predict(train_valid_x)

In [31]:
print(metrics.classification_report(train_valid_y, valid))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      9674
           1       0.97      0.95      0.96      9653

    accuracy                           0.96     19327
   macro avg       0.96      0.96      0.96     19327
weighted avg       0.96      0.96      0.96     19327

